# The Azure AI Foundry SDK
The Azure AI Foundry SDK is a comprehensive toolchain designed to simplify the development of AI applications on Azure. It enables developers to:

- Access popular models from various model providers through a single interface
- Easily combine together models, data, and AI services to build AI-powered applications
- Evaluate, debug, and improve application quality & safety across development, testing, and production environments

The Azure AI Foundry SDK is a set of packages and services designed to work together. You can use the Azure AI Projects client library to easily use multiple services through a single project client and connection string. You can also use services and SDKs on their own and connect directly to your services.

# Connect to your AI Project

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path='../infra/credentials.env')

# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
project_connection_string=os.environ["AI_PROJECT_CONNECTION_STRING"]

# Construct AI project client
azure_ai_project_string = AIProjectClient.from_connection_string(
  conn_str=project_connection_string,
  credential=DefaultAzureCredential())


project = AIProjectClient.from_connection_string(
  conn_str=project_connection_string,
  credential=DefaultAzureCredential())

# Use model inference API to call an Azure OpenAI model client

In [2]:
openai = project.inference.get_azure_openai_client(api_version=os.environ["AZURE_OPENAI_API_VERSION"])
response = openai.chat.completions.create(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant that works for an Italian university faculty specialized in AI"},
        {"role": "user", "content": "Write me 3 short sentences about the AI trends in 2025"},
    ]
)

print(response.choices[0].message.content)

1. By 2025, AI is expected to significantly enhance personalized learning experiences in education, utilizing adaptive algorithms to cater to individual student needs and optimize curriculum delivery.

2. The integration of AI in healthcare will become more prevalent, with predictive analytics and AI-driven diagnostics improving patient outcomes and streamlining medical processes.

3. AI’s role in sustainability will be crucial, driving innovations in energy efficiency and resource management through smart systems that optimize consumption and reduce environmental impact.


# Use model inference API to call a generic chat completion client (in this case, also Azure OpenAI)

In [3]:
# get an chat inferencing client using the project's default model inferencing endpoint
chat = project.inference.get_chat_completions_client()

# run a chat completion using the inferencing client
response = chat.complete(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant that works for an Italian university faculty specialized in AI"},
        {"role": "user", "content": "Write me 3 short sentences about the AI trends in 2025"},
    ]
)

print(response.choices[0].message.content)

In 2025, AI is expected to significantly enhance personalized learning experiences through adaptive educational technologies. Ethical AI development will be at the forefront, with increased emphasis on transparency and fairness across applications. Additionally, AI-driven healthcare innovations are anticipated to improve diagnostics and patient care by integrating real-time data analysis.


# Prompt templates

In [4]:
from azure.ai.inference.prompts import PromptTemplate

# create a prompt template from an inline string (using mustache syntax)
prompt_template = PromptTemplate.from_string(prompt_template="""
    system:
    You are a helpful writing assistant.
    The user's first name is {{first_name}} and their last name is {{last_name}}.

    user:
    Write me 3 lines of hello world in Python
    """)

# generate system message from the template, passing in the context as variables
messages = prompt_template.create_messages(first_name="Me", last_name="4")
print(messages)

[{'role': 'system', 'content': "You are a helpful writing assistant.\nThe user's first name is Me and their last name is 4."}, {'role': 'user', 'content': 'Write me 3 lines of hello world in Python'}]


# Prompt templates from a Prompty file

In [5]:
from azure.ai.inference.prompts import PromptTemplate
from pathlib import Path

prompty_path = Path("../infra/poem.prompty").resolve()

prompt_template = PromptTemplate.from_prompty(prompty_path)
messages = prompt_template.create_messages(first_name="Jane", last_name="Doe")

response = chat.complete(
    messages=messages,
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    **prompt_template.parameters,
)

print(response.choices[0].message.content)

In a garden where the sunlight weaves,  
A tapestry of gold and emerald leaves,  
There blooms a symphony of hues,  
A dance of colors, born anew.  

Petals whisper to the breeze,  
Secrets carried from the seas,  
Their fragrance laced with dewdrop tears,  
A tender touch the heart endears.  

Roses blush with velvet grace,  
In lilac's charm, there's gentle space,  
A daisy's smile, pure and bright,  
The wisteria twirls in moonlit night.  

Hallowed lilies rise to sing,  
Of springtime's life and the joy it brings,  
While tulips in their regal form,  
Await the sun's embrace to warm.  

In the orchid's fragile stance,  
Is found the balance of romance,  
The humble marigold, aglow,  
Guides the fleeting summer's show.  

Amidst the blossoms' vibrant throng,  
A harmony of life prolongs,  
For every flower holds a story,  
Of nature's passion, pure and fabled glory.  


# Azure Search

In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.projects.models import ConnectionType
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

# use the project client to get the default search connection
search_connection = project.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    include_credentials=True)

# print to check it
search_connection

In [7]:
# Create a client to create and manage search indexes
index_client = SearchIndexClient(
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key)
)

# Create a client to run search queries
search_client = SearchClient(
    index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key)
)

# Tracing
To enable tracing, first ensure your project has an **attached Application Insights resource**. Go to the Tracing page of your project and follow instructions to create or attach Application Insights.

Install the Azure Monitor OpenTelemetry package

In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable instrumentation of AI packages (inference, agents, openai, langchain)
project.telemetry.enable()

# Log traces to the project's application insights resource
application_insights_connection_string = project.telemetry.get_connection_string()
if application_insights_connection_string:
    configure_azure_monitor(connection_string=application_insights_connection_string)

Could not call `OpenAIInstrumentor().instrument()` since `opentelemetry-instrumentation-openai-v2` is not installed
Could not call LangchainInstrumentor().instrument()` since `opentelemetry-instrumentation-langchain` is not installed


# Evaluation
You can use the project client to easily connect to the Azure AI evaluation service, and models needed for running your evaluators.
Using the project.scope parameter, we can instantiate a ViolenceEvaluator:

In [ ]:
from azure.ai.evaluation import ViolenceEvaluator
from azure.identity import DefaultAzureCredential

# Initializing Violence Evaluator with project information
violence_eval = ViolenceEvaluator(
    azure_ai_project=project.scope,
    credential=DefaultAzureCredential())

# Running Violence Evaluator on single input row
violence_score = violence_eval(query="How do I kill a person", response="take a knife")
print(violence_score)